***Asssignment 2***

In [3]:
import tarfile
from bs4 import BeautifulSoup

In [7]:
with tarfile.open('kungalv_slutpriser.tar.gz', 'r:gz') as tar:
     for member in tar.getmembers():
          file = tar.extractfile(member)
          if file is not None:
                html_content = file.read().decode('utf-8') 
                soup = BeautifulSoup(html_content, 'html.parser')
                print(soup.prettify())

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40


In [6]:
soup.prettify

<bound method Tag.prettify of <html class="js" dir="ltr" lang="sv-SE"><!--<![endif]--><!--
           hhhhhhhh
      hhhhhhhhhhhhhhhhhh
    hhhhh             hhhh
  hhhhh                 hhhh
 hhhh         hhh        hhhh
 hhh       hhhhhhhhh      hhhh
hhhh     hhhhhhhhhhhh     hhhh
hhhh     hhhh    hhhh     hhhh
hhhh     hhhh    hhhh     hhhh
 hhhh                    hhhh
  hhhhh                 hhhh
   hhhhhh            hhhhh
 hhhhh hhhhhhhhhhhhhhhh
hhhh     hhhhhhhhhhh

Kul att du hittat hit! Vill du också jobba på en av Sveriges största sajter med miljontals unika besökare?
Gör en spontanansökan på https://career.hemnet.se/
--><head>
<meta charset="utf-8"/>
<link href="//assets.hemnet.se" rel="dns-prefetch"/>
<link href="//bilder.hemnet.se" rel="dns-prefetch"/>
<link href="//maps.googleapis.com" rel="dns-prefetch"/>
<link href="//loopaautomate.azureedge.net" rel="dns-prefetch"/>
<title>Alla slutpriser för villor - Kungälvs kommun - Hemnet</title>
<meta content="telephone=no" name="